TO DO

- is it possible to add captaincy bonus?
- add an optional constraint to allow only one defender from any given team (risk mitigation)?

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / 10
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 17
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 27 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.773752,2.257917,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,35.366118,0,3.552442
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.034415,0,2.540212
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.855909,0,1.868956
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.5,3.6,3.8,2.4,6.8,4.071,33.036595,3,3.036915
5,4.073752,1.357917,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.0,NaN,NaN,NaN,NaN,NaN,19.588862,1,1.473950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.853940,0,2.467211
583,42.800000,3.057143,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,1.4,2.4,7.8,2.0,6.400,31.711167,1,3.030370
611,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.627808,0,3.078863


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=5]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 27 prediction
id,,,,,,,,,,,,,,,,,,,,,
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,33.036595,3,3.036915
6,28.891000,2.407583,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,NaN,NaN,NaN,1.000000,3.600000,2.000000,24.618335,0,2.333258
7,12.918022,1.845432,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,-2.000000,NaN,NaN,2.332871,NaN,NaN,NaN,19.492307,1,1.731146
12,8.526730,1.705346,1,NaN,NaN,0.0,0.0,0.0,1.0,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,18.777711,0,1.709617
13,12.749168,1.593646,0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,1.000000,3.096585,1.000000,1.000000,1.000000,NaN,1.000000,18.488609,0,1.639799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,8.300000,1.037500,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1.000000,NaN,1.000000,NaN,NaN,1.300000,NaN,17.274031,0,1.639223
461,41.120925,3.163148,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,5.548812,4.418731,2.382085,NaN,5.048812,NaN,NaN,29.271696,0,2.692594
470,31.567409,3.507490,1,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.861196,28.811760,1,2.848831


In [5]:
optimize(df, 85.2, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 82.19999999999999
Expected points: 58.25641918182373


,element_type,web_name,now_cost,games played,gameweek 18 prediction
id,,,,,
559,1,Ramsdale,51,14.0,4.425128
127,2,Rüdiger,62,16.0,4.686001
135,2,Chilwell,57,6.0,5.086585
142,2,James,63,13.0,4.970903
256,2,Cancelo,68,16.0,5.023324
21,3,Smith Rowe,60,15.0,4.390820
233,3,Salah,131,17.0,5.334622
240,3,Jota,80,16.0,4.691434
250,3,Gündogan,73,12.0,5.005713


In [6]:
df = df.drop([135,4])
optimize(df, 85.2, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 81.19999999999999
Expected points: 57.459065675735474


,element_type,web_name,now_cost,games played,gameweek 18 prediction
id,,,,,
559,1,Ramsdale,51,14.0,4.425128
121,2,Thiago Silva,56,14.0,4.460551
127,2,Rüdiger,62,16.0,4.686001
142,2,James,63,13.0,4.970903
256,2,Cancelo,68,16.0,5.023324
21,3,Smith Rowe,60,15.0,4.390820
233,3,Salah,131,17.0,5.334622
240,3,Jota,80,16.0,4.691434
250,3,Gündogan,73,12.0,5.005713


In [8]:
df = df.drop([189])
optimize(df, 85.2, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 84.09999999999998
Expected points: 57.28050708770752


,element_type,web_name,now_cost,games played,gameweek 18 prediction
id,,,,,
559,1,Ramsdale,51,14.0,4.425128
127,2,Rüdiger,62,16.0,4.686001
142,2,James,63,13.0,4.970903
237,2,Alexander-Arnold,82,15.0,4.209063
256,2,Cancelo,68,16.0,5.023324
138,3,Mount,77,14.0,4.829995
21,3,Smith Rowe,60,15.0,4.390820
233,3,Salah,131,17.0,5.334622
240,3,Jota,80,16.0,4.691434


In [8]:
optimize(df, 87, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 80.10000000000001
Expected points: 57.47220706939697


,element_type,web_name,now_cost,games played,gameweek 18 prediction
id,,,,,
559,1,Ramsdale,51,14.0,4.425128
121,2,Thiago Silva,56,14.0,4.460551
127,2,Rüdiger,62,16.0,4.686001
142,2,James,63,13.0,4.970903
256,2,Cancelo,68,16.0,5.023324
21,3,Smith Rowe,60,15.0,4.411392
233,3,Salah,131,17.0,5.334622
240,3,Jota,80,16.0,4.691434
250,3,Gündogan,73,12.0,5.005713


In [ ]:
df = df.drop([259,262,228])
optimize(df, 85.2, f'gameweek {gameweek+1} prediction')

In [ ]:
optimize(df, 85.2, 'adjusted points per game')

In [ ]:
df = df.drop([268])
optimize(df, 85.2, 'adjusted points per game')

In [ ]:
#df = df.drop([119])
optimize(df, 85.2, 'xPoints_next10')